In [1]:
from xgboost import XGBClassifier, XGBRegressor
from tensorflow import keras
import tensorflow as tf
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
train_df['documentation remarks'].fillna(' ', inplace=True)
test_df['documentation remarks'].fillna(' ', inplace=True)
train_df['usability score'] = train_df['usability score'] / 10

In [3]:
train_df.head(20)

,dataset url,title,dataset author,dataset created,"file type, no. of files and file size",usability score,documentation remarks,votes,medal
0,https://www.kaggle.com/datasets/alejopaullier/...,New York City Weather Data 2019,moth,12 days ago,· 1 File (CSV) · 4 kB,1.00,"This dataset is well documented, overview is p...",20,Bronze
1,https://www.kaggle.com/datasets/muki2003/pytho...,Python Module whl Files,Code Breaker,13 days ago,· 24 Files (other) · 157 MB,0.88,The documentation for this dataset is almost c...,19,Bronze
2,https://www.kaggle.com/datasets/vencerlanz09/b...,Bottles Synthetic Images,VinceVence,5 days ago,"· 40002 Files (other, JSON) · 1 GB",0.88,The documentation for this dataset is almost c...,19,Bronze
3,https://www.kaggle.com/datasets/jpmiller/race-...,Horse Racing - Big Data Derby,JohnM,8 days ago,· 9 Files (CSV) · 40 MB,0.91,"This dataset is well documented, overview is p...",19,Bronze
4,https://www.kaggle.com/datasets/anasmahmood000...,Netflix Web-Series Dataset,Anas Mahmood,16 days ago,· 1 File (other) · 47 kB,0.65,This dataset is partially documented but is mi...,19,Bronze
5,https://www.kaggle.com/datasets/medaxone/fics-...,FICS Chess dataset 2021,Medaxone,a month ago,· 1 File (CSV) · 45 MB,0.91,"This dataset is well documented, overview is p...",19,Bronze
6,https://www.kaggle.com/datasets/andradaolteanu...,RSNA Fracture Detection,Andrada Olteanu,3 days ago,"· 711604 Files (other, CSV) · 4 GB",0.82,The documentation for this dataset is almost c...,19,Bronze
7,https://www.kaggle.com/datasets/rhtsingh/unive...,universal_image_embedding_src,torch,a month ago,· 12 Files (other) · 21 kB,0.69,This dataset is partially documented but is mi...,18,Bronze
8,https://www.kaggle.com/datasets/jirkaborovec/c...,Cervical Spine Fracture Detection: equalized PNG,Jirka Borovec,a month ago,· 17 GB,0.63,This dataset is partially documented but is mi...,17,Bronze
9,https://www.kaggle.com/datasets/yoshifumimiya/...,6 ovarian cancer datasets,Octopus210,19 days ago,"· 13 Files (other, CSV) · 220 MB",0.71,This dataset is partially documented but is mi...,17,Bronze


In [4]:
new_part = train_df['file type, no. of files and file size'].str.split(expand=True)
train_df[new_part.columns] = new_part
test_new_part = test_df['file type, no. of files and file size'].str.split(expand=True)
test_df[new_part.columns] = test_new_part
new_part.columns

RangeIndex(start=0, stop=9, step=1)

In [5]:
X = train_df[['votes', 'medal', 3, 6]]
y = train_df[['usability score']]
X

,votes,medal,3,6
0,20,Bronze,(CSV),kB
1,19,Bronze,(other),MB
2,19,Bronze,"(other,",1
3,19,Bronze,(CSV),MB
4,19,Bronze,(other),kB
...,...,...,...,...
7995,2,NaN,(CSV),MB
7996,2,NaN,(CSV),kB
7997,2,NaN,(CSV),kB
7998,2,NaN,(CSV),B


In [6]:
le = LabelEncoder()
le.fit(pd.concat([X['medal'], test_df['medal']], axis=0))
X['medal'] = le.transform(X['medal'])
le3 = LabelEncoder()
le3.fit(pd.concat([X[3], test_df[3]], axis=0))
X[3] = le3.transform(X[3])
le6 = LabelEncoder()
le6.fit(pd.concat([X[6], test_df[6]], axis=0))
X[6] = le6.transform(X[6])
X[0:50]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,votes,medal,3,6
0,20,0,1,84
1,19,0,6,83
2,19,0,7,0
3,19,0,1,83
4,19,0,6,84
5,19,0,1,83
6,19,0,7,44
7,18,0,6,84
8,17,0,8,86
9,17,0,7,23


In [7]:
train_sentences = train_df['documentation remarks'].to_list()
print(train_sentences)
test_sentences = test_df['documentation remarks'].to_list()
vocab_size = 10000
embedding_dim = 32
max_length = 20
trunc_type = 'post'
oov_token = '<OOV>'
tokenizer = keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)
padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length)
test_padded

['This dataset is well documented, overview is provided, files are in machine-readable formats and code examples are available in Kernels', 'The documentation for this dataset is almost complete but is missing file descriptions', 'The documentation for this dataset is almost complete but is missing file descriptions', 'This dataset is well documented, overview is provided, files are in machine-readable formats and code examples are available in Kernels', 'This dataset is partially documented but is missing machine-readable files, column descriptions and Kernels code examples', 'This dataset is well documented, overview is provided, files are in machine-readable formats and license is specified', 'The documentation for this dataset is almost complete but is missing file descriptions', 'This dataset is partially documented but is missing data collection source, machine-readable files and file descriptions', 'This dataset is partially documented but is missing license, file descriptions a

array([[ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  3, ..., 30,  2, 32],
       ...,
       [ 0,  0,  0, ..., 21, 19, 20],
       [ 0,  0,  3, ..., 30,  2, 32],
       [ 0,  0,  0, ...,  0,  0,  0]], dtype=int32)

In [8]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='Adam')
model.fit(padded, y, epochs=20)
X['documentation remarks'] = model.predict(padded)
test_df['documentation remarks'] = model.predict(test_padded)
X

Epoch 1/20
250/250 [==============================] - 3s 11ms/step - loss: 0.3840
Epoch 2/20
250/250 [==============================] - 3s 11ms/step - loss: 0.3577
Epoch 3/20
250/250 [==============================] - 3s 10ms/step - loss: 0.3577
Epoch 4/20
250/250 [==============================] - 3s 10ms/step - loss: 0.3577
Epoch 5/20
250/250 [==============================] - 3s 10ms/step - loss: 0.3577
Epoch 6/20
250/250 [==============================] - 2s 9ms/step - loss: 0.3577
Epoch 7/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3578
Epoch 8/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3576
Epoch 9/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3578
Epoch 10/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3576
Epoch 11/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3575
Epoch 12/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3576
Epoch 13/20
250/250 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,votes,medal,3,6,documentation remarks
0,20,0,1,84,0.987402
1,19,0,6,83,0.858681
2,19,0,7,0,0.858681
3,19,0,1,83,0.987402
4,19,0,6,84,0.753305
...,...,...,...,...,...
7995,2,1,1,83,0.865585
7996,2,1,1,84,0.865585
7997,2,1,1,84,0.865585
7998,2,1,1,81,0.865585


In [9]:
train_sentences = train_df['title'].to_list()
test_sentences = test_df['title'].to_list()
vocab_size = 10000
embedding_dim = 32
max_length = 20
trunc_type = 'post'
oov_token = '<OOV>'
tokenizer = keras.preprocessing.text.Tokenizer(num_words = vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(train_sentences)
test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_length)
padded = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_length)
test_padded

array([[   0,    0,    0, ...,  430, 1501,   33],
       [   0,    0,    0, ...,  712,  400,  101],
       [   0,    0,    0, ...,    4, 1198,    2],
       ...,
       [   0,    0,    0, ...,   10,   15, 1448],
       [   0,    0,    0, ..., 1741,    8,  382],
       [   0,    0,    0, ...,    0, 1502,    2]], dtype=int32)

In [10]:
model = keras.models.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Flatten(),
    keras.layers.Dense(16, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='Adam')
model.fit(padded, y, epochs=20)
X['title'] = model.predict(padded)
test_df['title'] = model.predict(test_padded)
X

Epoch 1/20
250/250 [==============================] - 2s 6ms/step - loss: 0.4105
Epoch 2/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3461
Epoch 3/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3374
Epoch 4/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3354
Epoch 5/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3346
Epoch 6/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3341
Epoch 7/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3338
Epoch 8/20
250/250 [==============================] - 1s 5ms/step - loss: 0.3338
Epoch 9/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3336
Epoch 10/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3335
Epoch 11/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3335
Epoch 12/20
250/250 [==============================] - 1s 6ms/step - loss: 0.3334
Epoch 13/20
250/250 [====

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


,votes,medal,3,6,documentation remarks,title
0,20,0,1,84,0.987402,0.996482
1,19,0,6,83,0.858681,0.889809
2,19,0,7,0,0.858681,0.887202
3,19,0,1,83,0.987402,0.918465
4,19,0,6,84,0.753305,0.656856
...,...,...,...,...,...,...
7995,2,1,1,83,0.865585,0.952959
7996,2,1,1,84,0.865585,0.936559
7997,2,1,1,84,0.865585,0.935514
7998,2,1,1,81,0.865585,0.918630


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [12]:
clf = keras.models.Sequential([
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
optimizer = keras.optimizers.Adam(lr=0.001)
clf.compile(loss='binary_crossentropy', optimizer='Adam')
clf.fit(X_train, y_train, epochs=10)

Epoch 1/10


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


188/188 [==============================] - 1s 2ms/step - loss: 0.4377
Epoch 2/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3872
Epoch 3/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3799
Epoch 4/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3743
Epoch 5/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3690
Epoch 6/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3651
Epoch 7/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3597
Epoch 8/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3560
Epoch 9/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3524
Epoch 10/10
188/188 [==============================] - 0s 2ms/step - loss: 0.3509


In [13]:
clf.predict(X_test[0:50])

2/2 [==============================] - 0s 5ms/step


array([[0.886054  ],
       [0.7107715 ],
       [0.9511555 ],
       [0.9508629 ],
       [0.8947442 ],
       [0.9086689 ],
       [0.938356  ],
       [0.92727435],
       [0.9522982 ],
       [0.92916805],
       [0.9246157 ],
       [0.9244343 ],
       [0.852257  ],
       [0.70476377],
       [0.8239714 ],
       [0.90840614],
       [0.7605769 ],
       [0.8535292 ],
       [0.9530512 ],
       [0.9062483 ],
       [0.94652313],
       [0.70626915],
       [0.92648745],
       [0.92766595],
       [0.9303762 ],
       [0.92894346],
       [0.6882117 ],
       [0.9463723 ],
       [0.98463106],
       [0.9326524 ],
       [0.8992776 ],
       [0.7903359 ],
       [0.7390318 ],
       [0.90047866],
       [0.9445691 ],
       [0.8771342 ],
       [0.8895568 ],
       [0.9101113 ],
       [0.92812353],
       [0.8737084 ],
       [0.91397995],
       [0.88850665],
       [0.8487778 ],
       [0.9291761 ],
       [0.92675203],
       [0.92675465],
       [0.74890184],
       [0.854

In [14]:
y_test[0:50]

,usability score
4265,0.88
4095,0.69
3817,1.00
4098,1.00
5688,0.94
374,0.94
4939,0.94
6036,1.00
2377,1.00
6580,1.00


In [16]:
test_df = test_df[['votes', 'medal', 3, 6, 'documentation remarks', 'title']]
test_df['medal'] = le.transform(test_df['medal'])
test_df[3] = le3.transform(test_df[3])
test_df[6] = le6.transform(test_df[6])
test_df['usability_score'] = clf.predict(test_df) * 10
test_df.drop(columns=['votes', 'medal', 3, 6], inplace=True)
test_df

16/16 [==============================] - 0s 2ms/step


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,documentation remarks,title,usability_score
0,0.865585,0.691412,7.617855
1,0.865585,0.999651,9.267088
2,0.945333,0.967880,9.278707
3,0.865585,0.883579,8.861549
4,0.945333,0.944429,9.263256
...,...,...,...
503,0.865585,0.699689,8.457557
504,0.865585,0.747999,8.029360
505,0.695947,0.689974,6.955252
506,0.945333,0.950765,9.333906


In [17]:
test_df.to_csv('output.csv', index=False)

In [ ]:
X